# AUC and Fairlay odds benchmarking

## Inputs

### Input files

In [1]:
MATCHES_DF = 'data/raw/premium_matches.2019-08-31.json.gz'
PREDICTIONS_DF = \
    'fitted/ti9_full_new_params_by_map_numba_fastmath.win_probs.gz'
SKILLS_DF = \
    'fitted/ti9_full_new_params_by_map_numba_fastmath.player_skills.gz'
FAIRLAY_PICKLE = 'data/external/dota2_markets.20190831.pkl'

### Parameters

In [2]:
# Scaling factor for logistic regression win probability.
LOGISTIC_SCALE = 3.0

# Maximum difference in minutes between the Fairlay closing datetime and the
# Datdota map start datetime.
MAX_TIMEDIFF_MINS = 1440

## Setup.

### Load libraries.

In [3]:
import os
while 'src' not in os.listdir():
    os.chdir("..")

In [4]:
import gzip
import json
import pickle

import numpy as np
import pandas as pd
import scipy

import src.fairlay
import src.load
import src.statics
import src.stats

### Load data.

#### Matches and predictions

In [5]:
with gzip.open(MATCHES_DF, 'rb') as fh:
    matches = src.load.MatchDF.from_json(json.load(fh)['data'])
pred_df = pd.read_csv(PREDICTIONS_DF, header=0, sep='\t', index_col=0)
skills_mat = pd.read_csv(SKILLS_DF, header=0, sep='\t', index_col=0)
skills_mat.columns = skills_mat.columns.astype(int)
match_pred = src.stats.MatchPred(matches, pred_df, LOGISTIC_SCALE, skills_mat)

#### Fairlay odds

1. Load the Fairlay data frame.
2. Merge Fairlay odds with match predictions.
3. Add columns for TI9 group stages and playoffs.

In [6]:
fairlay_df = pd.read_pickle(FAIRLAY_PICKLE)

In [7]:
fl_pred, fl_pred_sel = match_pred.merge_fairlay_df(fairlay_df,
                                                   MAX_TIMEDIFF_MINS)

In [8]:
comp_is_ti9 = fl_pred.Comp == 'Dota 2 - The International'
groupstage_date = fl_pred.startDate < '2019-08-18 12:00:00-04:00'
fl_pred_sel['ti9_group_stage'] = (comp_is_ti9 & groupstage_date).fillna(False)
fl_pred_sel['ti9_playoffs'] = (comp_is_ti9 & (~groupstage_date)).fillna(False)

## AUCs

In [9]:
ti9_group_stage_auc = match_pred._roc_auc(
    False,
    fl_pred.map_id[fl_pred_sel.ti9_group_stage])
ti9_playoffs_auc = match_pred._roc_auc(
    False,
    fl_pred.map_id[fl_pred_sel.ti9_playoffs])

print("TI9 group stage AUC: {:.3f}".format(ti9_group_stage_auc))
print("TI9 playoffs AUC: {:.3f}".format(ti9_playoffs_auc))

TI9 group stage AUC: 0.661
TI9 playoffs AUC: 0.592


## Fairlay odds

### Overall TI9, positive EV maps.

In [10]:
src.fairlay.fairlay_outcome_printer(
    fl_pred,
    fl_pred_sel.correct_match & fl_pred_sel.before_match & (fl_pred.ev > 0)
        & (fl_pred_sel.ti9_group_stage | fl_pred_sel.ti9_playoffs),
    "TI9 positive EV Fairlay predictions.")

src.fairlay.fairlay_outcome_printer(
    fl_pred,
    fl_pred_sel.correct_match & fl_pred_sel.before_match & (fl_pred.ev > 0.05)
        & (fl_pred_sel.ti9_group_stage | fl_pred_sel.ti9_playoffs),
    "TI9 EV > 0.05 Fairlay predictions.")

src.fairlay.fairlay_outcome_printer(
    fl_pred,
    fl_pred_sel.correct_match & fl_pred_sel.before_match & (fl_pred.ev > 0.1)
        & (fl_pred_sel.ti9_group_stage | fl_pred_sel.ti9_playoffs),
    "TI9 EV > 0.1 Fairlay predictions.")

src.fairlay.fairlay_outcome_printer(
    fl_pred,
    fl_pred_sel.correct_match & fl_pred_sel.before_match & (fl_pred.ev > 0.2)
        & (fl_pred_sel.ti9_group_stage | fl_pred_sel.ti9_playoffs),
    "TI9 EV > 0.2 Fairlay predictions.")

src.fairlay.fairlay_outcome_printer(
    fl_pred,
    fl_pred_sel.correct_match & fl_pred_sel.before_match & (fl_pred.ev > 0.3)
        & (fl_pred_sel.ti9_group_stage | fl_pred_sel.ti9_playoffs),
    "TI9 EV > 0.3 Fairlay predictions.")

TI9 positive EV Fairlay predictions.
------------------------------------
Maps included: 153
Total EV: 23.42 (-7.56, 51.64)
Total outcome: 18.328999999999997

TI9 EV > 0.05 Fairlay predictions.
----------------------------------
Maps included: 108
Total EV: 22.52 (-2.84, 50.44)
Total outcome: 20.509000000000004

TI9 EV > 0.1 Fairlay predictions.
---------------------------------
Maps included: 79
Total EV: 20.44 (-0.84, 46.88)
Total outcome: 16.986

TI9 EV > 0.2 Fairlay predictions.
---------------------------------
Maps included: 44
Total EV: 15.51 (-2.43, 36.69)
Total outcome: 7.255000000000001

TI9 EV > 0.3 Fairlay predictions.
---------------------------------
Maps included: 25
Total EV: 10.86 (-4.13, 26.91)
Total outcome: 6.774

